<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
launch_sites = [
    {"Launch Site": "CCAFS SLC-40 (Florida)", "Lat": 28.561857, "Long": -80.577366},
    {"Launch Site": "KSC LC-39A (Florida)",  "Lat": 28.608389, "Long": -80.604333},
    {"Launch Site": "VAFB SLC-4E (California)", "Lat": 34.632093, "Long": -120.610829},
    {"Launch Site": "Starbase (Boca Chica, TX)", "Lat": 25.997000, "Long": -97.156600},
]


center_lat = sum(s["Lat"] for s in launch_sites) / len(launch_sites)
center_lon = sum(s["Long"] for s in launch_sites) / len(launch_sites)
site_map = folium.Map(location=[center_lat, center_lon], zoom_start=4)

for s in launch_sites:
    coord = [s["Lat"], s["Long"]]
    name = s["Launch Site"]


    folium.Circle(
        location=coord,
        radius=1000,        
        fill=True,
        fill_opacity=0.1
    ).add_to(site_map)

    folium.Marker(
        location=coord,
        popup=name
    ).add_to(site_map)

    folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px; font-weight:600;">{name}</div>'
        )
    ).add_to(site_map)

site_map.save("task1_launch_sites_map.html")

site_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=center, zoom_start=5)

def assign_marker_color(outcome):
    
    return 'green' if outcome == 1 else 'red'


marker_cluster = MarkerCluster(name="Launch Outcomes")
site_map.add_child(marker_cluster)


for _, row in spacex_df.iterrows():
    lat, lon = float(row['Lat']), float(row['Long'])
    outcome = int(row['class']) 
    color = assign_marker_color(outcome)
    
    
    outcome_text = "Success" if outcome == 1 else "Failure"
    popup_html = (
        f"<b>Launch Site:</b> {row['Launch Site']}<br>"
        f"<b>Outcome:</b> {outcome_text}"
    )
    
    folium.Marker(
        location=[lat, lon],
        popup=popup_html,
        icon=folium.Icon(color=color, icon="rocket", prefix="fa")  
    ).add_to(marker_cluster)

folium.LayerControl().add_to(site_map)

site_map.save("task2_launch_outcomes_map.html")
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:

def assign_marker_color(outcome):
    """
    Assign a marker color based on the launch outcome.
    outcome: 1 -> green (success), 0 -> red (failure)
    """
    return 'green' if outcome == 1 else 'red'


spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
from folium.features import DivIcon

# launch_sites_df should contain unique ['Launch Site','Lat','Long'] (ساخته‌شده در Task 1)
for _, site in launch_sites_df.iterrows():
    lat, lon = float(site['Lat']), float(site['Long'])

    # Orange circle around site
    folium.Circle(
        location=[lat, lon],
        radius=800,              # می‌توانی 500–1200 تنظیم کنی
        color="#d35400",
        weight=3,
        fill=True,
        fill_color="#d35400",
        fill_opacity=0.12
    ).add_to(site_map)

    # Orange label next to the site
    folium.Marker(
        location=[lat, lon],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f"<div style='font-size:12px; color:#d35400; font-weight:700;'>{site['Launch Site']}</div>"
        )
    ).add_to(site_map)

site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [15]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math
import pandas as pd
import folium

# --- Haversine distance helper (returns distance in km) ---
def haversine_km(coord1, coord2):
    R = 6371.0088  # Earth's radius in kilometers
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    return 2 * R * math.asin(math.sqrt(a))

# --- Simulated proximity data (to replace missing CSV) ---
data = [
    {"Name": "Atlantic Coast", "Lat": 28.50, "Long": -80.60, "Category": "coastline"},
    {"Name": "Railway",        "Lat": 28.52, "Long": -80.58, "Category": "railway"},
    {"Name": "Highway 528",    "Lat": 28.54, "Long": -80.62, "Category": "highway"},
    {"Name": "Cape Canaveral", "Lat": 28.40, "Long": -80.70, "Category": "city"},
]
geo_df = pd.DataFrame(data)

# --- Choose a launch site (from launch_sites_df) ---
site_row = launch_sites_df.head(1)  # Pick the first launch site
site_lat, site_lon = float(site_row.iloc[0]['Lat']), float(site_row.iloc[0]['Long'])
site_name = site_row.iloc[0]['Launch Site']
site_coord = (site_lat, site_lon)

# --- Find nearest point in each category ---
categories = ['coastline', 'railway', 'highway', 'city']
nearest_points = {}

for cat in categories:
    points = geo_df[geo_df['Category'] == cat]
    min_dist, min_point = float('inf'), None
    for _, row in points.iterrows():
        dist = haversine_km((row['Lat'], row['Long']), site_coord)
        if dist < min_dist:
            min_dist, min_point = dist, row
    if min_point is not None:
        nearest_points[cat] = {
            'lat': min_point['Lat'],
            'lon': min_point['Long'],
            'distance_km': min_dist,
            'name': min_point['Name']
        }

# --- Draw map ---
site_map = folium.Map(location=[site_lat, site_lon], zoom_start=12)

# Mark the launch site
folium.Marker(
    [site_lat, site_lon],
    popup=f"{site_name}",
    icon=folium.Icon(color='blue', icon='flag')
).add_to(site_map)

# Add nearest features and draw lines
for cat, info in nearest_points.items():
    pt = [info['lat'], info['lon']]
    label = f"{info['name']} ({cat}) — {info['distance_km']:.2f} km"

    # Marker for feature
    folium.Marker(
        pt,
        popup=label,
        icon=folium.Icon(color='purple', icon='info-sign')
    ).add_to(site_map)

    # Line from site to feature
    folium.PolyLine(
        [[site_lat, site_lon], pt],
        weight=4, opacity=0.7,
        tooltip=f"{cat.title()} distance: {info['distance_km']:.2f} km"
    ).add_to(site_map)

# Fit bounds
site_map.fit_bounds([[site_lat, site_lon]] + [[v['lat'], v['lon']] for v in nearest_points.values()])

# Show map
site_map


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [18]:
import math
import folium

# Haversine distance (km)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0088  # Earth's radius (km)
    φ1, λ1, φ2, λ2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dφ, dλ = φ2 - φ1, λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1) * math.cos(φ2) * math.sin(dλ/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# --- Launch site coordinates ---
# If you already have them from Task 3 selection:
launch_site_lat = float(site_lat)   # or: float(site_row.iloc[0]['Lat'])
launch_site_lon = float(site_lon)   # or: float(site_row.iloc[0]['Long'])

# --- Replace these with the coastline point you read from MousePosition ---
# e.g., Lat: 28.56367  Lon: -80.57163
coastline_lat = 28.56367
coastline_lon = -80.57163

# Compute distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)  # km

print(f"Distance to coastline: {distance_coastline:.2f} km")

# --- Draw the coastline point and a line on the existing map ---
# Coastline marker
folium.CircleMarker(
    location=[coastline_lat, coastline_lon],
    radius=6,
    fill=True,
    fill_opacity=0.9,
    popup=f"Coastline point\n({coastline_lat:.5f}, {coastline_lon:.5f})"
).add_to(site_map)

# Polyline from launch site to coastline
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=4, opacity=0.8,
    tooltip=f"Coastline distance: {distance_coastline:.2f} km"
).add_to(site_map)

# Optionally re-display the map to see the updates
site_map


Distance to coastline: 0.58 km


In [19]:
from folium.features import DivIcon

# Coordinates of your selected coastline point (replace with MousePosition values)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance (if not already done)
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon, coastline_lat, coastline_lon
)

# Create and add a text marker showing the distance
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % 
             ("{:10.2f} KM".format(distance_coastline)),
    ),
)

# Add the distance marker to the map
site_map.add_child(distance_marker)

# Display the updated map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [21]:
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

lines = folium.PolyLine(
    locations=coordinates,
    weight=1
)

site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [22]:
from folium.features import DivIcon

# Example coordinates for closest features (replace these with actual MousePosition picks)
closest_city_lat, closest_city_lon = 28.4, -80.7
closest_railway_lat, closest_railway_lon = 28.52, -80.58
closest_highway_lat, closest_highway_lon = 28.54, -80.62

# Store in a dictionary for easy looping
features = {
    "City": (closest_city_lat, closest_city_lon),
    "Railway": (closest_railway_lat, closest_railway_lon),
    "Highway": (closest_highway_lat, closest_highway_lon),
}

# Loop through each feature, calculate distance, add marker and line
for feature_name, (lat, lon) in features.items():
    # Calculate distance
    distance = calculate_distance(launch_site_lat, launch_site_lon, lat, lon)

    # Create a marker with distance text
    distance_marker = folium.Marker(
        location=[lat, lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' %
                 ("{:10.2f} KM".format(distance)),
        ),
        popup=f"{feature_name}: {distance:.2f} km"
    )
    site_map.add_child(distance_marker)

    # Draw a line from the launch site to the feature
    line = folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [lat, lon]],
        weight=2,
        color="blue",
        opacity=0.7
    )
    site_map.add_child(line)

# Display updated map
site_map



After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
